# Requirement

Please do same exercise using two different framework.

Framework 1. pandas
framework 2. apache beam python https://beam.apache.org/documentation/sdks/python/


using two input files dataset1 and dataset2 

join dataset1 with dataset2 and get tier

generate below output file

legal_entity, counterparty, tier, max(rating by counterparty), sum(value where status=ARAP), sum(value where status=ACCR)

Also create new record to add total for each of legal entity, counterparty & tier.

Sample data:
legal_entity, counterparty, tier, max(rating by counterparty), sum(value where status=ARAP), sum(value where status=ACCR)
L1,Total, Total, calculated_value, calculated_value,calculated_value
L1, C1, Total,calculated_value, calculated_value,calculated_value
Total,C1,Total,calculated_value, calculated_value,calculated_value
Total,Total,1,calculated_value, calculated_value,calculated_value
L2,Total,Total,calculated_value, calculated_value,calculated_value
....
like all other values.

where caluclated_value in sample data needs to be calculated using above method.

## Initialize

In [2]:
from typing import List
import pandas as pd

## Helper methods

1. *load_and_merge_datasets* to merge two datasets provided on column counter_party
2. *process_data* to process dataframes for
    - Calculate the maximum rating by counter_party (after grouping on "legal_entity", "counter_party", "tier")
    - Calculate the sum of values where status is "ARAP" (after grouping on "legal_entity", "counter_party", "tier")
    - Calculate the sum of values where status is "ACCR" (after grouping on "legal_entity", "counter_party", "tier")
    - Calculate the total count of values (after grouping on "legal_entity", "counter_party", "tier")
3. *merge_dataframes* to combine the above 4 dataframes into one (after grouping on "legal_entity", "counter_party", "tier")

In [3]:
def load_and_merge_datasets()->pd.DataFrame:
    df_dataset1 = pd.read_csv("data\\dataset1.csv")
    df_dataset2 = pd.read_csv("data\\dataset2.csv")
    
    # Merge the datasets on the 'counter_party' column using a left join
    merged_df = pd.merge(left=df_dataset1, right=df_dataset2, on="counter_party", how="left")
    
    return merged_df

def process_data(df:pd.DataFrame) -> List[pd.DataFrame]:
    # Select the required columns
    df = df[["legal_entity", "counter_party", "tier", "rating", "value", "status"]]
    
    # Define group-by columns
    group_by_columns = ["legal_entity", "counter_party", "tier"]
    
    # Calculate the maximum rating by counter_party
    max_rating = df.groupby(group_by_columns)["rating"].max().reset_index()
    max_rating.rename(columns={"rating": "max(rating by counterparty)"}, inplace=True)
    
    # Calculate the sum of values where status is "ARAP"
    sum_arap = df[df["status"] == "ARAP"].groupby(group_by_columns)["value"].sum().reset_index()
    sum_arap.rename(columns={"value": "sum(value where status=ARAP)"}, inplace=True)
    
    # Calculate the sum of values where status is "ACCR"
    sum_accr = df[df["status"] == "ACCR"].groupby(group_by_columns)["value"].sum().reset_index()
    sum_accr.rename(columns={"value": "sum(value where status=ACCR)"}, inplace=True)
    
    # Calculate the total count of values
    count_values = df.groupby(group_by_columns)["value"].count().reset_index()
    count_values.rename(columns={"value": "Total(count with same legal_entity, counter_party,tier)"}, inplace=True)
    
    return max_rating, sum_arap, sum_accr, count_values

def merge_dataframes(dataframes: List[pd.DataFrame], on: List) -> pd.DataFrame:
    merged = None
    for dataframe in dataframes:
        if merged is not None:
            merged = merged.merge(right=dataframe, on=on, how="outer")
        else:
            merged = dataframe
    return merged

## Main program starts from here

1. First we merge the datasets

In [5]:
    merged_df = load_and_merge_datasets()

    display(merged_df)

,invoice_id,legal_entity,counter_party,rating,status,value,tier
0,1,L1,C1,1,ARAP,10,1
1,2,L2,C2,2,ARAP,20,2
2,3,L3,C3,4,ACCR,30,3
3,4,L1,C4,6,ARAP,40,4
4,5,L2,C5,4,ACCR,50,5
5,6,L3,C6,6,ACCR,60,6
6,7,L1,C1,2,ARAP,10,1
7,8,L2,C2,3,ACCR,40,2
8,9,L3,C3,3,ACCR,80,3
9,10,L1,C4,5,ACCR,100,4


2. Then we do processing onver dataframe:
     - Max of rating (for each legal_entity, counter_party, tier)
     - Sum of values where status is ARAP (for each legal_entity, counter_party, tier)
     - Sum of values where status is ACCR (for each legal_entity, counter_party, tier)
     - Total count of values (for each legal_entity, counter_party, tier)

In [7]:
    max_rating, sum_arap, sum_accr, count_values = process_data(merged_df)
    dataframes = [max_rating, sum_arap, sum_accr, count_values]

    for dataframe in dataframes:
        display(dataframe)

,legal_entity,counter_party,tier,max(rating by counterparty)
0,L1,C1,1,3
1,L1,C3,3,6
2,L1,C4,4,6
3,L2,C2,2,3
4,L2,C3,3,2
5,L2,C5,5,6
6,L3,C3,3,4
7,L3,C6,6,6


,legal_entity,counter_party,tier,sum(value where status=ARAP)
0,L1,C1,1,40
1,L1,C3,3,5
2,L1,C4,4,40
3,L2,C2,2,20
4,L2,C5,5,1000
5,L3,C6,6,145


,legal_entity,counter_party,tier,sum(value where status=ACCR)
0,L1,C4,4,100
1,L2,C2,2,40
2,L2,C3,3,52
3,L2,C5,5,115
4,L3,C3,3,145
5,L3,C6,6,60


,legal_entity,counter_party,tier,"Total(count with same legal_entity, counter_party,tier)"
0,L1,C1,1,3
1,L1,C3,3,1
2,L1,C4,4,2
3,L2,C2,2,2
4,L2,C3,3,1
5,L2,C5,5,3
6,L3,C3,3,3
7,L3,C6,6,3


## final result after merging

In [8]:
    result = merge_dataframes(dataframes, on=["legal_entity", "counter_party", "tier"])
    
    # Fill missing values with 0 and drop duplicates
    result.fillna(0, inplace=True)
    result.drop_duplicates(inplace=True)

    display(result)

,legal_entity,counter_party,tier,max(rating by counterparty),sum(value where status=ARAP),sum(value where status=ACCR),"Total(count with same legal_entity, counter_party,tier)"
0,L1,C1,1,3,40.0,0.0,3
1,L1,C3,3,6,5.0,0.0,1
2,L1,C4,4,6,40.0,100.0,2
3,L2,C2,2,3,20.0,40.0,2
4,L2,C3,3,2,0.0,52.0,1
5,L2,C5,5,6,1000.0,115.0,3
6,L3,C3,3,4,0.0,145.0,3
7,L3,C6,6,6,145.0,60.0,3
